In [46]:
import numpy as np
import pandas as pd
import scipy.sparse
import xgboost as xgb
import mix_pandas as mix
import predict as predict_mix
import db_column_name as db

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 10, 10


cn = db.ColumnName()

target_minT = pd.read_csv('./data/31286_103.csv')
mix.set_index_date(target_minT, cn.date)

X = pd.read_csv('./data/character_31286.csv')
mix.set_index_date(X, cn.date)

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or 
       x == cn.offset]]

X = X[X[cn.offset] == 69]
X = X[X.index.hour == 21]
print(X.shape)

# X = mix.mean_day(X)
# target_minT.index = target_minT.index.round('D')

X = X.drop([cn.offset], axis=1)

target_minT = target_minT.reindex(X.index)
target_minT = mix.clean(target_minT)
X = X.reindex(target_minT.index)
X = mix.clean(X)
print(X.shape)

target_minT = target_minT.iloc[3:] # remove on change

target_minT = mix.winsorized(target_minT, cn.value, [0.05, 0.95], 5)
X = X.reindex(target_minT.index)
print(X.shape)

(1245, 55)
(1195, 54)
(979, 54)


In [47]:
corr = X.corr()
upper = corr.where(~np.tril(np.ones(corr.shape)).astype(np.bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
print(to_drop)
X_drop = X.drop(to_drop, axis=1)
X_drop.head()

['avg tdd925', 'avg t925', 'avg slp', 'avg rainnc', 'avg t500', 'avg advTdd850', 'avg td2', 'avg td925', 'avg advRh850', 'avg t850', 'avg slptend', 'avg td850']


,avg td500,avg rh925,avg v10,avg wdir850,avg snownc,avg defV500,avg t2,avg u850,avg wdir10,avg u10,...,avg defSt500,avg w500,avg rainc,avg wspd850,avg w850,avg dudy500,avg u500,avg rot500,avg advRh2,avg tdd500
actual_date,,,,,,,,,,,,,,,,,,,,,
2014-06-20 21:00:00,-16.3832,57.4869,-2.137750,13.4974,0.000000,0.000080,16.79120,-3.64175,316.370,2.041520,...,-0.000035,-0.063531,7.72829,15.61320,0.031968,0.000009,-2.82198,-0.000025,-0.000769,1.97802
2014-10-28 21:00:00,-53.2110,33.1061,-3.023350,45.9554,0.000000,0.000033,-15.38900,-4.11547,333.192,1.512220,...,0.000009,-0.104186,0.00000,5.96798,0.037972,-0.000037,-11.33590,0.000080,-0.003336,29.46220
2014-10-30 21:00:00,-38.1176,54.4846,-0.650154,306.2130,0.011389,0.000084,-22.30960,14.13860,286.141,1.419810,...,-0.000017,0.026363,0.00000,17.51580,0.045806,-0.000013,21.18470,-0.000053,-0.000783,6.96273
2014-11-01 21:00:00,-34.8647,84.8004,-2.232900,121.9540,0.049867,0.000057,-9.84371,-3.49329,25.601,-1.058090,...,0.000029,0.001384,0.00000,4.17964,0.035269,-0.000038,12.20310,0.000043,-0.000395,5.30676
2014-11-02 21:00:00,-43.3544,93.3471,-0.492943,135.1090,4.404100,0.000027,-6.28352,-3.26421,215.005,0.362182,...,-0.000015,-0.016323,0.00000,4.66506,0.019166,-0.000013,5.90434,0.000040,-0.000135,7.17927


In [48]:
params = {
    'verbosity':0,
    'max_depth': 4,
    
    'min_child_weight': 6,
#     'learning_rate': 0.03,
}

reg = xgb.XGBRegressor(**params)

predict = predict_mix.predict_model_split(reg, X_drop, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()

Train size 164
Mean squared error 0.35584
Mean absolute error 0.37387
Median absolute error 0.23496
Test size 163
Mean squared error 20.48908
Mean absolute error 3.58482
Median absolute error 3.00597

Train size 327
Mean squared error 0.49539
Mean absolute error 0.51687
Median absolute error 0.34488
Test size 163
Mean squared error 41.60720
Mean absolute error 4.52323
Median absolute error 2.69972

Train size 490
Mean squared error 0.98423
Mean absolute error 0.74821
Median absolute error 0.58799
Test size 163
Mean squared error 78.61994
Mean absolute error 7.53598
Median absolute error 7.80517

Train size 653
Mean squared error 1.52361
Mean absolute error 0.92347
Median absolute error 0.70515
Test size 163
Mean squared error 11.38432
Mean absolute error 2.71740
Median absolute error 2.30700

Train size 816
Mean squared error 1.99223
Mean absolute error 1.07309
Median absolute error 0.84294
Test size 163
Mean squared error 13.83361
Mean absolute error 2.98357
Median absolute error 2.68

In [49]:
from sklearn.feature_selection import SelectFromModel

importances = pd.DataFrame(reg.feature_importances_, index=X_drop.columns, columns=['Score'])
importances = importances.sort_values(by=['Score'], ascending=False)


# xgb.plot_importance(reg, importance_type='gain')

slice_importances = importances.iloc[:10]
X_select = X.loc[:, slice_importances.index]
X_select.head()

,avg t2,avg snownc,avg lfc,avg rh925,avg rh2,avg wdir10,avg wdir850,avg cape,avg lcl,avg totalprecip
actual_date,,,,,,,,,,
2014-06-20 21:00:00,16.79120,0.000000,2659.22,57.4869,73.5068,316.370,13.4974,124.557,1807.27,7.728290
2014-10-28 21:00:00,-15.38900,0.000000,18037.10,33.1061,73.8041,333.192,45.9554,-0.100,6391.33,0.000000
2014-10-30 21:00:00,-22.30960,0.011389,17846.50,54.4846,100.0000,286.141,306.2130,-0.100,3854.87,0.011414
2014-11-01 21:00:00,-9.84371,0.049867,17805.30,84.8004,89.4535,25.601,121.9540,-0.100,3296.84,0.050112
2014-11-02 21:00:00,-6.28352,4.404100,17782.20,93.3471,92.6628,215.005,135.1090,-0.100,3319.24,4.449360


In [53]:
params = {
    'verbosity':0,
    'max_depth': 4,
    
    'min_child_weight': 6,
#     'learning_rate': 0.03,
}

reg = xgb.XGBRegressor(**params)

predict = predict_mix.predict_model_split(reg, X_select, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()

Train size 164
Mean squared error 1.61642
Mean absolute error 0.91602
Median absolute error 0.62827
Test size 163
Mean squared error 18.93922
Mean absolute error 3.54645
Median absolute error 3.40293

Train size 327
Mean squared error 2.60247
Mean absolute error 1.23208
Median absolute error 0.98649
Test size 163
Mean squared error 39.21593
Mean absolute error 5.05717
Median absolute error 4.17526

Train size 490
Mean squared error 2.88572
Mean absolute error 1.26970
Median absolute error 0.94459
Test size 163
Mean squared error 74.40296
Mean absolute error 7.38590
Median absolute error 7.69245

Train size 653
Mean squared error 2.83277
Mean absolute error 1.27068
Median absolute error 0.96064
Test size 163
Mean squared error 10.04559
Mean absolute error 2.53691
Median absolute error 2.23977

Train size 816
Mean squared error 3.07954
Mean absolute error 1.33712
Median absolute error 1.01247
Test size 163
Mean squared error 12.47088
Mean absolute error 2.77639
Median absolute error 2.24

In [51]:
from sklearn.preprocessing import PolynomialFeatures

pl = PolynomialFeatures(degree=2, include_bias=False)
X_polynomial = pd.DataFrame(pl.fit_transform(X_select), index=X.index)
X_polynomial.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
actual_date,,,,,,,,,,,,,,,,,,,,,
2014-06-20 21:00:00,16.79120,0.000000,2659.22,57.4869,73.5068,316.370,13.4974,124.557,1807.27,7.728290,...,182.179807,1681.195652,2.439345e+04,104.311821,15514.446249,225108.12939,962.612618,3.266225e+06,13967.106668,59.726466
2014-10-28 21:00:00,-15.38900,0.000000,18037.10,33.1061,73.8041,333.192,45.9554,-0.100,6391.33,0.000000,...,2111.898789,-4.595540,2.937161e+05,0.000000,0.010000,-639.13300,-0.000000,4.084910e+07,0.000000,0.000000
2014-10-30 21:00:00,-22.30960,0.011389,17846.50,54.4846,100.0000,286.141,306.2130,-0.100,3854.87,0.011414,...,93766.401369,-30.621300,1.180411e+06,3.495115,0.010000,-385.48700,-0.001141,1.486002e+07,43.999486,0.000130
2014-11-01 21:00:00,-9.84371,0.049867,17805.30,84.8004,89.4535,25.601,121.9540,-0.100,3296.84,0.050112,...,14872.778116,-12.195400,4.020628e+05,6.111298,0.010000,-329.68400,-0.005011,1.086915e+07,165.209598,0.002511
2014-11-02 21:00:00,-6.28352,4.404100,17782.20,93.3471,92.6628,215.005,135.1090,-0.100,3319.24,4.449360,...,18254.441881,-13.510900,4.484592e+05,601.148580,0.010000,-331.92400,-0.444936,1.101735e+07,14768.493686,19.796804


In [55]:
params = {
    'verbosity':0,
    'max_depth': 4,
    'gamma': 6    
}

reg = xgb.XGBRegressor(**params)
predict = predict_mix.predict_model_split(reg, X_polynomial, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()

Train size 164
Mean squared error 0.84978
Mean absolute error 0.69136
Median absolute error 0.53032
Test size 163
Mean squared error 23.00920
Mean absolute error 3.99641
Median absolute error 3.94240

Train size 327
Mean squared error 1.32404
Mean absolute error 0.87712
Median absolute error 0.62583
Test size 163
Mean squared error 44.67165
Mean absolute error 5.71722
Median absolute error 5.55702

Train size 490
Mean squared error 1.79001
Mean absolute error 1.01814
Median absolute error 0.82745
Test size 163
Mean squared error 70.14656
Mean absolute error 7.11762
Median absolute error 7.50531

Train size 653
Mean squared error 2.16928
Mean absolute error 1.11616
Median absolute error 0.87957
Test size 163
Mean squared error 8.93611
Mean absolute error 2.38100
Median absolute error 2.04043

Train size 816
Mean squared error 2.30018
Mean absolute error 1.17226
Median absolute error 0.94836
Test size 163
Mean squared error 14.54845
Mean absolute error 3.03745
Median absolute error 2.562